In [1]:
from utils.env import make_doom_env
import gym

num_envs = 1

# Initializing environment
envs = gym.vector.SyncVectorEnv(
    [make_doom_env(level_config_path='vizdoom/scenarios/basic.cfg') for i in range(num_envs)]
)


In [6]:
# Testing observations by rendering  
# from matplotlib import pyplot as plt
# import cv2

# frame = envs.reset();
# plt.imshow(cv2.cvtColor(frame[0], cv2.COLOR_BGR2RGB))

from gym.spaces import Discrete

# print(isinstance(envs.single_action_space, Discrete))
print(envs.reset().shape)

(1, 4, 120, 160)


In [2]:
from torch.utils.tensorboard import SummaryWriter
from agents.ppo_agent import PpoAgent
from utils.memory import Memory
from utils.time import current_timestamp_ms
import time

# Setting up agent
agent = PpoAgent(envs.single_observation_space, envs.single_action_space)

# Setting up agent training config
global_step = 0
start_time = time.time()
num_steps = 256
num_mini_batches = 1
batch_size = int(num_envs * num_steps)
mini_batch_size = int(batch_size / num_mini_batches)
total_timesteps = 10000000
num_updates = int(total_timesteps / batch_size)
memory = Memory(agent.device, num_steps, num_envs, envs.single_observation_space.shape, envs.single_action_space.shape)

# Setting up debugging for Tensorboard
tensorboard_writer = SummaryWriter(f"logs/log_{current_timestamp_ms()}")

In [8]:
print(mini_batch_size)

512


In [3]:
import torch
import numpy as np

observation = torch.Tensor(envs.reset()).to(agent.device)
done = torch.zeros(num_envs).to(agent.device)

for update in range(1, num_updates + 1):
    # Calculating learning rate annealing coefficient
    learning_rate_anneal_coef = 1.0 - (update - 1.0) / num_updates

    for step in range(0, num_steps):
        global_step += num_envs

        # Getting next action and it's value
        with torch.no_grad():
            action, log_prob, _, value = agent.get_optimal_action_and_value(observation)
            value = value.flatten()

        observation_, reward, done_, info = envs.step(action.cpu().numpy())

        # Saving experience in memory
        memory.remember(
            step=step, 
            observation= observation,
            action=action,
            value=value,
            log_prob=log_prob,
            reward=torch.tensor(np.array(reward, dtype=np.float32)).to(agent.device).view(-1),
            done=done
        )

        # Saving new observation and done status for next step
        observation = torch.Tensor(observation_).to(agent.device) 
        done =  torch.Tensor(done_).to(agent.device)

        # Writing step debug info to TensorBoard
        for item in info:
            if "episode" in item.keys():
                print(f"global_step={global_step}, episodic_return={item['episode']['r']}")
                tensorboard_writer.add_scalar("charts/episodic_return", item["episode"]["r"], global_step)
                tensorboard_writer.add_scalar("charts/episodic_length", item["episode"]["l"], global_step)
                break

    training_stats = agent.train(
        memory.observations,
        memory.actions,
        memory.log_probs,
        memory.rewards,
        memory.values,
        memory.dones,
        batch_size,
        learning_rate_anneal_coef=learning_rate_anneal_coef,
        mini_batch_size=mini_batch_size
    )

    print("SPS:", int(global_step / (time.time() - start_time)))

    tensorboard_writer.add_scalar("charts/learning_rate", training_stats.learning_rate, global_step)
    tensorboard_writer.add_scalar("losses/value_loss", training_stats.value_loss, global_step)
    tensorboard_writer.add_scalar("losses/policy_loss", training_stats.policy_loss, global_step)
    tensorboard_writer.add_scalar("losses/entropy_loss", training_stats.entropy_loss, global_step)
    tensorboard_writer.add_scalar("charts/old_approx_kl", training_stats.old_approx_kl, global_step)
    tensorboard_writer.add_scalar("charts/approx_kl", training_stats.approx_kl, global_step)
    tensorboard_writer.add_scalar("charts/clip_fraction", training_stats.clip_fraction, global_step)
    tensorboard_writer.add_scalar("charts/explained_variance", training_stats.explained_variance, global_step)
    tensorboard_writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)


global_step=66, episodic_return=-216.0
global_step=134, episodic_return=-219.0
global_step=136, episodic_return=95.0
global_step=141, episodic_return=83.0
global_step=216, episodic_return=-375.0
global_step=233, episodic_return=21.0
SPS: 11
global_step=279, episodic_return=-121.0
global_step=282, episodic_return=91.0
global_step=357, episodic_return=-370.0
global_step=383, episodic_return=-26.0
global_step=401, episodic_return=17.0
global_step=476, episodic_return=-370.0
SPS: 14
global_step=551, episodic_return=-375.0
global_step=626, episodic_return=-365.0
global_step=636, episodic_return=58.0
global_step=639, episodic_return=91.0
global_step=654, episodic_return=33.0
global_step=656, episodic_return=95.0
global_step=670, episodic_return=38.0
global_step=745, episodic_return=-345.0
SPS: 15
global_step=820, episodic_return=-370.0
global_step=895, episodic_return=-370.0
global_step=968, episodic_return=-248.0
global_step=973, episodic_return=83.0
SPS: 16
global_step=1048, episodic_retur

KeyboardInterrupt: 

In [ ]:
# Saving models
agent.save_models()

In [4]:
# Closing environments
envs.close()
# Closing tensorboard writer
tensorboard_writer.close()